In [ ]:
import time
import numpy as np
import matplotlib.pyplot as plt
# plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams["font.size"] = "8"
from matplotlib import rc

from control_interface_v2 import real_hand_v2

### Joint tracking tests
connect all motors and test how the parameters will affect the tracking performance

In [ ]:
leap_hand = real_hand_v2(path_suffix='../', motor_nums=24)

In [ ]:
q_real = leap_hand.q
print(q_real)
leap_hand.send_traj(np.zeros(24))

In [ ]:
np.array(
            [[-1,-1.4,-1.4,-1.4] * 5,
             [1,1.4,1.4,1.4] * 5])

In [ ]:
from scipy import signal
f=1
signal.sawtooth(2 * np.pi * f * (0 + 1/f/4), 0.5)

In [ ]:

q0 = np.zeros(24)
t0 = time.time()
ref='sin'
# ref='tri'
i = 0
f = 0.5
# a = np.array([0.4, 0.4,0.4,0.4]*5)
a = np.array([0.4, 0.4,0.4,0.4])
q_records = []
while 1:
    q_cmd = np.copy(q0)
    vel_cmd = np.zeros(24)
    acc_cmd = np.zeros(24)
    t = time.time() - t0
    if ref =='sin':
        q_i = a * np.sin(2 * np.pi *f *t)
        vel_cmd[:len(a)] = a * np.cos(2 * np.pi *f *t) * 2 * np.pi * f 
        acc_cmd[:len(a)] =  - a * np.sin(2 * np.pi *f *t) * (2 * np.pi * f )**2
    else:
        q_i = a * signal.sawtooth(2 * np.pi * f * (t + 1/f/4), 0.5)
        q_i = np.clip(q_i, -0.2,0.2)
    q_cmd[i: i+len(a)] = q_i
    leap_hand.move_to(q_cmd, vel=np.abs(vel_cmd))
    # leap_hand.move_to(q_cmd)
    q_real = leap_hand.q
    # q_records.append(np.array([t, q_i, q_real[i: i+len(a)]]))
    q_records.append(np.concatenate([np.array([t]), q_i, q_real[i: i+len(a)], leap_hand.ddq, leap_hand.dq]))
    # q_records.append(np.array([t, q_i, 0]))
    # time.sleep(0.01)
    if t > 5 * 1/f:
        break

In [ ]:
q_records = np.vstack(q_records)
fig_width = 90/25.4  * 0.7
import matplotlib
matplotlib.use('Agg')


error = []
for i in range(1,  1+ len(a)):
    fig, ax = plt.subplots(figsize=(fig_width, fig_width))
    ax.plot(q_records[:,0], q_records[:,i], color='r',label='Ref')
    ax.plot(q_records[:,0], q_records[:,i+len(a)], color='b',label='Real')
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Joint position (rad)')
    ax.legend(loc='upper right')
    
    ax.set_title('Tracking performance test, joint ' + str(i))
    fig.savefig(  str(i) + '_joint_test.jpg',format='jpg', bbox_inches='tight',  pad_inches=0.0, dpi=600)
    
    # print(str(i), 'mean tracking error:',   np.mean(np.abs(q_records[:,i] - q_records[:,i+len(a)]))/a[i-1])
    error.append(np.mean(np.abs(q_records[:,i] - q_records[:,i+len(a)]))/a[i-1])
print(np.mean(error))

In [ ]:
q_records.shape

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
t = np.linspace(0, 5, 500)
fig_width = 90/25.4  * 0.7

i= 0
fig, ax = plt.subplots(figsize=(fig_width, fig_width))
ax.plot(q_records[:,0], q_records[:,i+20], color='r',label='Ref')

plt.show()

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
t = np.linspace(0, 5, 500)
f=0.5
triangle = signal.sawtooth(2 * np.pi * f * (t + 1/f /4), 0.5)
triangle = np.clip(triangle, -0.5,0.5)
plt.plot(t, triangle)
plt.show()

In [ ]:
dt = q_records[1:,0] - q_records[:-1,0]

fig, ax = plt.subplots(figsize=(fig_width, fig_width))
ax.plot(dt, color='r')
np.mean(dt)